<a href="https://colab.research.google.com/github/johnwesleyharding/DS-Unit-1-Sprint-2-Statistics/blob/master/JWH_assignment_DS_122_Sampling_Confidence_Intervals_and_Hypothesis_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment - Build a confidence interval

A confidence interval refers to a neighborhood around some point estimate, the size of which is determined by the desired p-value. For instance, we might say that 52% of Americans prefer tacos to burritos, with a 95% confidence interval of +/- 5%.

52% (0.52) is the point estimate, and +/- 5% (the interval $[0.47, 0.57]$) is the confidence interval. "95% confidence" means a p-value $\leq 1 - 0.95 = 0.05$.

In this case, the confidence interval includes $0.5$ - which is the natural null hypothesis (that half of Americans prefer tacos and half burritos, thus there is no clear favorite). So in this case, we could use the confidence interval to report that we've failed to reject the null hypothesis.

But providing the full analysis with a confidence interval, including a graphical representation of it, can be a helpful and powerful way to tell your story. Done well, it is also more intuitive to a layperson than simply saying "fail to reject the null hypothesis" - it shows that in fact the data does *not* give a single clear result (the point estimate) but a whole range of possibilities.

How is a confidence interval built, and how should it be interpreted? It does *not* mean that 95% of the data lies in that interval - instead, the frequentist interpretation is "if we were to repeat this experiment 100 times, we would expect the average result to lie in this interval ~95 times."

For a 95% confidence interval and a normal(-ish) distribution, you can simply remember that +/-2 standard deviations contains 95% of the probability mass, and so the 95% confidence interval based on a given sample is centered at the mean (point estimate) and has a range of +/- 2 (or technically 1.96) standard deviations.

Different distributions/assumptions (90% confidence, 99% confidence) will require different math, but the overall process and interpretation (with a frequentist approach) will be the same.

Your assignment - using the data from the prior module ([congressional voting records](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)):


### Confidence Intervals:
1. Generate and numerically represent a confidence interval
2. Graphically (with a plot) represent the confidence interval
3. Interpret the confidence interval - what does it tell you about the data and its distribution?




In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats
from scipy.stats import chisquare, t, normaltest, kruskal, stats
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel, ttest_1samp

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

df = pd.read_csv('house-votes-84.data', 
                 header=None,
                 names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa'])

df = df.replace({'?': np.NaN, 'n': 0, 'y': 1})

dem = df[df['party'] == "democrat"]
rep = df[df['party'] == "republican"]

--2019-10-09 15:25:25--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.7’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2019-10-09 15:25:26 (286 KB/s) - ‘house-votes-84.data.7’ saved [18171/18171]



In [0]:
def confidence_interval(data, confidence=0.95):
  """
  Calculate a confidence interval around a sample mean for given data.
  Using t-distribution and two-tailed test, default 95% confidence. 
  
  Arguments:
    data - iterable (list or numpy array) of sample observations
    confidence - level of confidence for the interval
  
  Returns:
    tuple of (mean, lower bound, upper bound)
  """
  data = np.array(data)
  mean = np.mean(data)
  n = len(data)
  stderr = stats.sem(data)
  margin = stderr * stats.t.ppf((1 + confidence) / 2.0, n - 1)
  return (mean, mean - margin, mean + margin)

In [4]:
(smean, cimin, cimax) = confidence_interval(df['south-africa'].dropna(), confidence = .95)
print(f'Confidence Interval: {cimin} to {cimax} \n')

AttributeError: ignored

In [0]:
# def showconfidence(series):

#   (smean, cimin, cimax) = confidence_interval(df[series].dropna(), confidence = .95)
  
#   sns.distplot(df[series].dropna())
#   plt.axvline(x = cimin, color = 'o')
#   plt.axvline(x = cimax, color = 'o')
#   plt.axvline(x = smean, color = 'g')
#   plt.show()
  
#   print(f'Confidence Interval: {cimin} to {cimax} \n')

In [0]:
# showconfidence(df['south-africa'])

### Chi-squared tests:
4. Take a dataset that we have used in the past in class that has **categorical** variables. Pick two of those categorical variables and run a chi-squared tests on that data
  - By hand using Numpy
  - In a single line using Scipy

In [0]:
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

!unzip LoanStats_2018Q4.csv.zip

df = pd.read_csv('LoanStats_2018Q4.csv', header = 1, skipfooter = 2)

In [7]:
df.describe()

id  member_id  ...  settlement_percentage  settlement_term
count  0.0        0.0  ...             168.000000       168.000000
mean   NaN        NaN  ...              52.028333        18.101190
std    NaN        NaN  ...               9.760939         6.851408
min    NaN        NaN  ...              40.000000         1.000000
25%    NaN        NaN  ...              45.000000        14.000000
50%    NaN        NaN  ...              45.010000        18.000000
75%    NaN        NaN  ...              60.000000        24.000000
max    NaN        NaN  ...              98.240000        24.000000

[8 rows x 108 columns]

In [6]:
df.describe(include = object)

term int_rate  ... settlement_status settlement_date
count       128412   128412  ...               168             168
unique           2       46  ...                 3               8
top      36 months   13.56%  ...            ACTIVE        Aug-2019
freq         88179     6974  ...               152              57

[4 rows x 36 columns]

In [12]:
observed = pd.crosstab(df['home_ownership'], df['installment']).values
print(observed.shape)

(4, 12363)


In [13]:
chi_squared, p_value, dof, expected = stats.chi2_contingency(observed)

print(f"Chi-Squared: {chi_squared}")
print(f"P-value: {p_value}")
print(f"Degrees of Freedom: {dof}") 
print("Expected: \n", np.array(expected))

AttributeError: ignored

Scipy failed to mask my lack of statistics knowledge.  I have no idea what's wrong.

Other Questions

It was mentioned a couple times today that standard deviations and t-statistic are not the same thing.  As I understand it, standard deviation is some kind of measure of the distribution of data in a sample or population.  Is t-statistic something different?  Or a different measurement of that distribution?  What kind of features would the data have to easily observe that difference.

I'm lost on the significance of picking a null hypothesis, a confidence interval, and identifying an observation that really means something.  It seems like any way of considering the data is either obvious, insignificant, or irrelevant.  Is it all about who agrees with your perception of the null hypothesis?  Do you always want the smallest confidence interval you can achieve or one that's consistent across a domain?



## Stretch goals:

1. Write a summary of your findings, mixing prose and math/code/results. *Note* - yes, this is by definition a political topic. It is challenging but important to keep your writing voice *neutral* and stick to the facts of the data. Data science often involves considering controversial issues, so it's important to be sensitive about them (especially if you want to publish).
2. Apply the techniques you learned today to your project data or other data of your choice, and write/discuss your findings here.
3. Refactor your code so it is elegant, readable, and can be easily run for all issues.

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)